In [1]:
import gdal
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
#Lists all subdatasets of any one file
file = gdal.Open('AOD_Nepal_china/MOD04_L2.A2022163.0125.061.2022163134820.hdf')
#
print(file)
#file = gdal.Open('/Users/nimishmishra/Downloads/Kanpur MODIS Data/November/2018.11.18.hdf')
for path, desc in file.GetSubDatasets():
    print(desc)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000022458A00E70> >
[203x135] Scan_Start_Time mod04 (64-bit floating-point)
[203x135] Solar_Zenith mod04 (16-bit integer)
[203x135] Solar_Azimuth mod04 (16-bit integer)
[203x135] Sensor_Zenith mod04 (16-bit integer)
[203x135] Sensor_Azimuth mod04 (16-bit integer)
[203x135] Scattering_Angle mod04 (16-bit integer)
[203x135] Land_sea_Flag mod04 (16-bit integer)
[4060x2708] Aerosol_Cldmask_Land_Ocean mod04 (16-bit integer)
[4060x2708] Cloud_Pixel_Distance_Land_Ocean mod04 (16-bit integer)
[203x135] Land_Ocean_Quality_Flag mod04 (16-bit integer)
[203x135] Optical_Depth_Land_And_Ocean mod04 (16-bit integer)
[203x135] Image_Optical_Depth_Land_And_Ocean mod04 (16-bit integer)
[203x135] Average_Cloud_Pixel_Distance_Land_Ocean mod04 (16-bit integer)
[203x135] Aerosol_Type_Land mod04 (16-bit integer)
[203x135] Fitting_Error_Land mod04 (16-bit integer)
[3x203x135] Surface_Reflectance_Land mod04 (16-bit integer)
[3x203x13

In [3]:
# Opens the HDF file
def load_data(FILEPATH):
    ds = gdal.Open(FILEPATH)
    return ds

In [4]:
# Opens the data HDF file and returns as a dataframe
def read_dataset(SUBDATASET_NAME, FILEPATH):
    dataset = load_data(FILEPATH)
    path = ''
    for sub, description in dataset.GetSubDatasets():
        if (description.endswith(SUBDATASET_NAME)):
            path = sub
            break
    if(path == ''):
        print(SUBDATASET_NAME + ' not found')
        return
    subdataset = gdal.Open(path)
    subdataset = subdataset.ReadAsArray()
    subdataset = pd.DataFrame(subdataset)
    return subdataset

In [5]:
# Loads the HDF file, gets the Latitude subdataset and returns the information of the nearest pixel to specified position
def find_latitude_position(CITY_LATITUDE, FILEPATH):
    dataset_to_load = 'Latitude (32-bit floating-point)'
    latitude_dataframe = read_dataset(dataset_to_load, FILEPATH)
    min_diff = 100
    row_number = -1
    column_number = -1
    rows = latitude_dataframe.shape[0]
    columns = latitude_dataframe.shape[1]
    for i in range(rows):
        for j in range(columns):
            lat = latitude_dataframe.iloc[i][j]
            diff = np.abs(lat - CITY_LATITUDE)
            if(diff < min_diff):
                min_diff = diff
                row_number = i
                column_number = j
                found_lat = lat
    if(row_number == -1 or column_number == -1):
        print("Latitude not found. You might have chosen wrong scene")
    return latitude_dataframe, row_number, column_number


In [6]:
# Loads the HDF file, gets the Longitude subdataset and returns the information of the nearest pixel to specified position
def find_longitude_position(CITY_LONGITUDE, LATITUDE_ROW_NUMBER, FILEPATH):
    dataset_to_load = 'Longitude (32-bit floating-point)'
    longitude_dataframe = read_dataset(dataset_to_load, FILEPATH)
    min_diff = 100
    row_number = -1
    column_number = -1
    rows = longitude_dataframe.shape[0]
    columns = longitude_dataframe.shape[1]
    for j in range(columns):
        lon = longitude_dataframe.iloc[LATITUDE_ROW_NUMBER][j]
        diff = np.abs(lon - CITY_LONGITUDE)
        if(diff < min_diff):
            min_diff = diff
            row_number = LATITUDE_ROW_NUMBER
            column_number = j
            found_lon = lon
    if(column_number == -1):
        print("Longitude not found. You might have chosen wrong scene")
    return longitude_dataframe, column_number

In [7]:
def find_product_value(LATITUDE_ROW_NUMBER, LONGITUDE_COLUMN_NUMBER, SUBDATASET, FILEPATH):
    dataset = read_dataset(SUBDATASET, FILEPATH)
    if(LATITUDE_ROW_NUMBER < 0 or LONGITUDE_COLUMN_NUMBER < 0):
        return -1
    return dataset.iloc[LATITUDE_ROW_NUMBER][LONGITUDE_COLUMN_NUMBER]

In [8]:
# Creates a list of values of the product under observation across all datasets
def create_list(PRODUCT, CITY_LATITUDE, CITY_LONGITUDE, FILEPATH):
    latitude_dataframe, lat_row, lat_column = find_latitude_position(CITY_LATITUDE, FILEPATH)
    if lat_row == -1:
        return -1
    longitude_dataframe, lon_column = find_longitude_position(CITY_LONGITUDE, lat_row, FILEPATH)
    city_row_number = lat_row
    city_column_number = lon_column
    if(PRODUCT == 'AOD'):
        subdataset = 'Deep_Blue_Aerosol_Optical_Depth_550_Land mod04 (16-bit integer)'
    elif(PRODUCT == 'Scattering Angle'):
        subdataset = 'Scattering_Angle mod04 (16-bit integer)'
    elif(PRODUCT == 'Cloud Fraction'):
        subdataset = 'Deep_Blue_Cloud_Fraction_Land (16-bit integer)'
    elif(PRODUCT == 'Combined'):
        subdataset = 'AOD_550_Dark_Target_Deep_Blue_Combined (16-bit integer)'
    elif(PRODUCT == 'Angstrom Exponent'):
        subdataset = 'Deep_Blue_Angstrom_Exponent_Land (16-bit integer)'
    row_begin = city_row_number - 1
    row_end = city_row_number + 1
    column_begin = city_column_number - 1
    column_end = city_column_number + 1
    total = 0
    for i in range(row_begin, row_end, 1):
        for j in range(column_begin, column_end, 1):
            pixel_value = find_product_value(i, j, subdataset, FILEPATH)
            if(pixel_value == -9999):
                pixel_value = 0
            total = total + pixel_value
    product_value = total / 9
    return product_value, latitude_dataframe.iloc[lat_row][lat_column], longitude_dataframe.iloc[lat_row][lon_column]

In [9]:
import glob
# Defining lists that we want
scattering_angle = []
deep_blue_AOD_Land = []
cloud_fraction = []
file_names = []
latitude = []
longitude = []
combined_AOD = []
angstrom_exponent = []
directory = 'AOD_Nepal_china' # Put here the month directory from which to extract files

long_N = 88.8
lat_n = 27.26

for file in glob.glob(directory + '/*.hdf'):
    FILEPATH = file
    print(FILEPATH)
    try:
        scattering_angle_value, lat, lon = create_list('Scattering Angle',  lat_n,long_N, FILEPATH)
        deep_blue_AOD_Land_value, lat, lon = create_list('AOD', lat_n,long_N, FILEPATH)
        cloud_fraction_value, lat, lon = create_list('Cloud Fraction', lat_n,long_N, FILEPATH)
        combined_AOD_value, lat, lon = create_list('Combined', lat_n,long_N, FILEPATH)
        angstrom_exponent_value, lat, lon = create_list('Angstrom Exponent', lat_n,long_N, FILEPATH)
        print("Data collected for: " + str(lat) + " N (%f N for Nepal) and "%lat_n + str(lon) + " E.(%f for Nepal)"%long_N)
        file_names.append(file)
        scattering_angle.append(scattering_angle_value)
        deep_blue_AOD_Land.append(deep_blue_AOD_Land_value)
        latitude.append(lat)
        longitude.append(lon)
        cloud_fraction.append(cloud_fraction_value)
        combined_AOD.append(combined_AOD_value)
        angstrom_exponent.append(angstrom_exponent_value)
    except:
        pass

final_list = pd.DataFrame()
final_list['File Name'] = pd.Series(file_names)
final_list['Scattering_angle'] = pd.Series(scattering_angle)
final_list['Deep Blue AOD'] = pd.Series(deep_blue_AOD_Land)
final_list['Cloud Fraction'] = pd.Series(cloud_fraction)
final_list['Combined AOD'] = pd.Series(combined_AOD)
final_list['Angstrom Exponent'] = pd.Series(angstrom_exponent)
final_list['Latitude'] = pd.Series(latitude)
final_list['Longitude'] = pd.Series(longitude)
path = 'AOD_Nepal_china/2013_2023_PM.csv'
final_list.to_csv(path)

AOD_Nepal_china\MOD04_L2.A2022163.0125.061.2022163134820.hdf
Data collected for: 48.680912 N (27.260000 N for Kanpur) and 125.769684 E.(88.800000 for Kanpur)
AOD_Nepal_china\MOD04_L2.A2022163.0130.061.2022163134822.hdf
Data collected for: 31.845228 N (27.260000 N for Kanpur) and 124.0733 E.(88.800000 for Kanpur)
AOD_Nepal_china\MOD04_L2.A2022163.0135.061.2022163134741.hdf
Data collected for: 27.259996 N (27.260000 N for Kanpur) and 123.0712 E.(88.800000 for Kanpur)
AOD_Nepal_china\MOD04_L2.A2022163.0305.061.2022163134847.hdf
Data collected for: 45.004723 N (27.260000 N for Kanpur) and 100.8082 E.(88.800000 for Kanpur)
AOD_Nepal_china\MOD04_L2.A2022163.0310.061.2022163134827.hdf
Data collected for: 27.961449 N (27.260000 N for Kanpur) and 98.93048 E.(88.800000 for Kanpur)
AOD_Nepal_china\MOD04_L2.A2022163.0315.061.2022163134714.hdf
Data collected for: 27.259998 N (27.260000 N for Kanpur) and 98.39059 E.(88.800000 for Kanpur)
AOD_Nepal_china\MOD04_L2.A2022163.0320.061.2022163134755.hdf
D

In [10]:
def plot(x_axis, y_axis_1, y_axis_2, label1, label2, directory, month):
    figure = plt.figure()
    ax = sns.lineplot(x=date, y=y_axis_1)
    ax = sns.lineplot(x=x_axis, y=y_axis_2)
    figure.legend(labels = [label1, label2])
    plt.show()
    path = (directory + month + '/') + month + label1 + label2 + '.png'
    fig = ax.get_figure()
    fig.savefig(path)

In [11]:
final_list

,File Name,Scattering_angle,Deep Blue AOD,Cloud Fraction,Combined AOD,Angstrom Exponent,Latitude,Longitude
0,AOD_Nepal_china\MOD04_L2.A2022163.0125.061.202...,3114.111111,31.333333,25.333333,-0.222222,333.111111,48.680912,125.769684
1,AOD_Nepal_china\MOD04_L2.A2022163.0130.061.202...,3209.444444,-0.222222,222.000000,-0.222222,-0.222222,31.845228,124.073303
2,AOD_Nepal_china\MOD04_L2.A2022163.0135.061.202...,3216.333333,-0.222222,222.000000,-0.222222,-0.222222,27.259996,123.071198
3,AOD_Nepal_china\MOD04_L2.A2022163.0305.061.202...,3144.111111,4.111111,-0.222222,-0.222222,333.111111,45.004723,100.808197
4,AOD_Nepal_china\MOD04_L2.A2022163.0310.061.202...,3215.777778,-0.222222,222.000000,-0.222222,-0.222222,27.961449,98.930481
5,AOD_Nepal_china\MOD04_L2.A2022163.0315.061.202...,3216.222222,-0.222222,222.000000,-0.222222,-0.222222,27.259998,98.390587
6,AOD_Nepal_china\MOD04_L2.A2022163.0320.061.202...,1584.777778,-0.333333,110.777778,-0.333333,-0.333333,13.240330,96.025154
7,AOD_Nepal_china\MOD04_L2.A2022163.0445.061.202...,7174.000000,58.555556,153.333333,18.444444,333.333333,41.266273,88.739502
8,AOD_Nepal_china\MOD04_L2.A2022163.0450.061.202...,6009.333333,0.000000,444.444444,0.000000,0.000000,27.260290,88.804161
9,AOD_Nepal_china\MOD04_L2.A2022163.0455.061.202...,2931.333333,-0.222222,222.000000,-0.222222,-0.222222,27.164173,88.863220
